<a href="https://colab.research.google.com/github/fangyihao/llm-tutorials/blob/main/Rotary_Position_Embedding/VideoRoPE_test.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!git clone https://github.com/fangyihao/llm-tutorials

Cloning into 'llm-tutorials'...
remote: Enumerating objects: 114, done.
remote: Counting objects: 100% (99/99), done.
remote: Compressing objects: 100% (86/86), done.
remote: Total 114 (delta 38), reused 23 (delta 4), pack-reused 15 (from 1)
Receiving objects: 100% (114/114), 3.34 MiB | 13.86 MiB/s, done.
Resolving deltas: 100% (42/42), done.


In [2]:
%cd llm-tutorials/Rotary_Position_Embedding/

/content/llm-tutorials/Rotary_Position_Embedding


In [6]:
from transformers import AutoProcessor, AutoModelForVision2Seq
from modeling_videorope import Qwen2VLForConditionalGeneration
import torch
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"

processor = AutoProcessor.from_pretrained("Qwen/Qwen2-VL-2B-Instruct")
model = Qwen2VLForConditionalGeneration.from_pretrained("Qwen/Qwen2-VL-2B-Instruct", torch_dtype=torch.bfloat16,
                                                 _attn_implementation="eager", device_map="auto")
                                              #  _attn_implementation="flash_attention_2" if DEVICE == "cuda" else "eager").to(DEVICE)

from PIL import Image
from transformers.image_utils import load_image


# Load images
image1 = load_image("https://huggingface.co/spaces/HuggingFaceTB/SmolVLM/resolve/main/example_images/rococo.jpg")
image2 = load_image("https://huggingface.co/spaces/HuggingFaceTB/SmolVLM/resolve/main/example_images/rococo_1.jpg")

# Create input messages
messages = [
    {
        "role": "user",
        "content": [
            {"type": "image"},
            {"type": "image"},
            {"type": "text", "text": "Can you describe the two images?"}
        ]
    },
]

# Prepare inputs
prompt = processor.apply_chat_template(messages, add_generation_prompt=True)
inputs = processor(text=prompt, images=[image1, image2], return_tensors="pt")
inputs = inputs.to(DEVICE)


# Generate outputs
generated_ids = model.generate(**inputs, max_new_tokens=500)
generated_texts = processor.batch_decode(
    generated_ids,
    skip_special_tokens=True,
)

print(generated_texts[0])


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

which_rope='m_rope'
system
You are a helpful assistant.
user
Can you describe the two images?
assistant
The first image shows a painting of a woman in a red dress holding flowers. The second image shows a painting of a ship in the ocean.
